# EDA : Brazilian E-Commerce Public Dataset

---

### 1. 데이터 전처리

In [1]:
### 데이터 조인을 통한 다수 데이터셋 통합 진행 
### 아래 SQL 쿼리문 참고, order_id는 중복 허용됨, order_status에서 cancel된 주문은 제외

# create or replace view order_info as
# select a.order_id,
# 	b.customer_id,
#   d.customer_unique_id,
# 	d.customer_city,
# 	d.customer_state,
# 	d.customer_zip_code_prefix as customer_zipcode,
# 	c.product_category_name_english as category,
# 	b.order_status,
# 	b.order_purchase_timestamp as purchase_date,
# 	b.order_estimated_delivery_date as estimated_delivery_date,
# 	datediff(b.order_estimated_delivery_date, b.order_purchase_timestamp) as estimated_delivery_period,
# 	a.price + a.freight_value as total_price
# from olist_order_items_dataset a
# left join olist_orders_dataset b on a.order_id = b.order_id
# left join product_dataset c on a.product_id = c.product_id
# left join olist_customers_dataset d on b.customer_id = d.customer_id
# having b.order_status != 'canceled';

In [2]:
import mysql.connector
import pandas as pd
import plotly_express as px
from datetime import datetime, timedelta

# mysql 데이터베이스 연결
conn = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "cx@1076044150",
    database = "olist"
)
cursor = conn.cursor(buffered=True)


# 쿼리한 데이터셋 불러오기
sql = 'select * from order_info'
cursor.execute(sql)
result = cursor.fetchall()
columns = [col[0] for col in cursor.description]
df = pd.DataFrame(data=result, columns=columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112108 entries, 0 to 112107
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   order_id                   112108 non-null  object        
 1   customer_id                112108 non-null  object        
 2   customer_unique_id         112108 non-null  object        
 3   customer_city              112108 non-null  object        
 4   customer_state             112108 non-null  object        
 5   customer_zipcode           112108 non-null  int64         
 6   category                   110497 non-null  object        
 7   order_status               112108 non-null  object        
 8   purchase_date              112108 non-null  datetime64[ns]
 9   estimated_delivery_date    112108 non-null  datetime64[ns]
 10  estimated_delivery_period  112108 non-null  int64         
 11  total_price                112108 non-null  float64 

---

### 2. 재구매 고객 및 상품 카테고리 분석
- 재구매 기준 : olist몰의 판매제품의 사용연수가 다양한 종합몰이므로, 최초 구매 시기로부터 1년을 유효기간으로 가정하여 재구매를 정의
- 고객이 최초 구매 후 재구매한 주문건수 분석 (구매일이 1일을 경과하지 않으면 재구매로 보지 않는다.)

In [3]:
# 재구매 고객 현황 (order_id 중복 제외)
df_re = pd.pivot_table(data=df, index='order_id', values=['customer_unique_id', 'purchase_date'], aggfunc='first')
df_re.sort_values(by='purchase_date', ascending=True, inplace=True)

# 고객별 최초 구매일 그룹핑
result = df_re.groupby('customer_unique_id')['purchase_date'].min()

# 재구매 여부 확인
df_re['repurchase'] = df_re.apply(lambda row: 
                                  ((row['purchase_date'] - result[row['customer_unique_id']]).days >= 1) and
                                  ((row['purchase_date'] - result[row['customer_unique_id']]).days <= 730), axis=1)

print('재구매된 주문건수 : ', len(df_re[df_re['repurchase'] == True]), '건 (구매일이 1일이상 경과하지 않으면 재구매로 보지 않는다.)')

재구매된 주문건수 :  2285 건 (구매일이 1일이상 경과하지 않으면 재구매로 보지 않는다.)


In [4]:
# 재구매 카테고리 현황 (재구매 대상 order_id만 필터링)
df_re2 = df[df['order_id'].isin(df_re[df_re['repurchase'] == True].index)]

# 재구매 카테고리 구매빈도
df_re2 = pd.pivot_table(data=df_re2, index='category', values='customer_id', aggfunc='count')
df_re2 = df_re2.sort_values(by='customer_id', ascending=False).rename(columns={'customer_id':'order_count'})
df_re2 = df_re2[df_re2['order_count'] > 1]

px.bar(df_re2, x=df_re2.index, y='order_count',
       labels={"category":"카테고리","order_count":"주문건수"}, 
       title='카테고리별 재구매 주문건수')

In [5]:
df = pd.merge(df, df_re['repurchase'], on='order_id', how='left')

---

- 재구매 전환율이 가장 높은 카테고리는 'diapers_and_hygiene'로 15.15%의 재구매 전환율을 보였다.
- 가장 재구매가 많은 카테고리는 'bed_bath_table'로 구매 수요도 많아서 최초 구매 건수도 많아 전환율은 약 3.6%를 보였다.
- 전환율과 각 특성과의 상관계수가 낮은 것으로 보아, 최초 구매 카테고리와 동일한 카테고리의 제품을 재구매한다고 보기 어렵다.  

In [6]:
# 카테고리별 재구매 전환율이 높은 카테고리 분석
df_re3 = pd.pivot_table(data=df, index='category', columns='repurchase', aggfunc='size', fill_value=0)
df_re3['conversion_rate'] = round(df_re3[True] / df_re3[False], 4)
df_re3.sort_values(by='conversion_rate', ascending=False, inplace=True)
df_re3.rename(columns={True:'re_purchase_cnt', False:'1st_purchase_cnt'}, inplace=True)
df_re3.head(10)

repurchase,1st_purchase_cnt,re_purchase_cnt,conversion_rate
category,,,
diapers_and_hygiene,33,5,0.1515
fashio_female_clothing,44,4,0.0909
drinks,353,25,0.0708
fashion_bags_accessories,1921,104,0.0541
dvds_blu_ray,59,3,0.0508
tablets_printing_image,79,4,0.0506
arts_and_craftmanship,23,1,0.0435
costruction_tools_tools,99,4,0.0404
bed_bath_table,10711,386,0.0360


In [7]:
# 상관분석 결과
df_re3.corr()

repurchase,1st_purchase_cnt,re_purchase_cnt,conversion_rate
repurchase,,,
1st_purchase_cnt,1.000000,0.965378,0.012319
re_purchase_cnt,0.965378,1.000000,0.083423
conversion_rate,0.012319,0.083423,1.000000


- 최초구매 + 재구매 카테고리의 총 매출이 높은 상위 10개 카테고리
- 마케팅 1회 비용에 대한 신규 고객 유입 관련 데이터가 없으나, 총 매출량이 높은 카테고리 위주의 마케팅을 진행하는 것이 효율적이다.

In [8]:
# 품목별 매출 분석
df_re4 = pd.pivot_table(data=df, index='category', values='total_price', aggfunc='sum')
df_re4 = pd.merge(df_re3, df_re4, on='category', how='left')
df_re4.sort_values(by='total_price', ascending=False).head(10)

,1st_purchase_cnt,re_purchase_cnt,conversion_rate,total_price
category,,,,
health_beauty,9431,203,0.0215,1437665.78
watches_gifts,5805,165,0.0284,1298292.47
bed_bath_table,10711,386,0.0360,1240386.13
sports_leisure,8324,266,0.0320,1147244.63
computers_accessories,7591,190,0.0250,1050941.58
furniture_decor,8057,241,0.0299,899626.04
housewares,6724,191,0.0284,772035.14
cool_stuff,3728,52,0.0139,704176.47
auto,4126,79,0.0191,678652.60


---

### 3. CLV (Customer Lifetime Value, 고객생애가치)
- CLV를 모델의 평가 기준으로 고려
- 신규고객 유치 및 기존 고객 마케팅에 들어간 비용은 olist에서 확인이 어려우므로 무시
- CLV = 평균구매액 × 평균구매빈도

In [9]:
# olist에서 확인 가능한 데이터로 CLV를 산정
df['year'] = df['purchase_date'].dt.year
df_clv1 = pd.pivot_table(data=df, index='customer_unique_id', values=['total_price', 'year', 'order_id'], 
                         aggfunc={'total_price':'sum', 'year':'first', 'order_id':'count'})
df_clv1.rename(columns={'order_id':'purchase_cnt'}, inplace=True)

df_clv2 = pd.pivot_table(data=df_clv1, index='year', values=['total_price', 'purchase_cnt'], aggfunc='mean')
df_clv2.rename(columns={'total_price':'avg_price', 'purchase_cnt':'avg_purchase_cnt'}, inplace=True)
df_clv2['clv'] = df_clv2['avg_price'] * df_clv2['avg_purchase_cnt']

df_clv3 = pd.pivot_table(data=df_clv1, index='year', values=['total_price', 'purchase_cnt'], aggfunc={'total_price':'sum', 'purchase_cnt':'count'})
df_clv3.rename(columns={'purchase_cnt':'customer_unique_id_cnt'}, inplace=True)
df_clv = pd.merge(df_clv2, df_clv3, on='year', how='left')
df_clv

,avg_purchase_cnt,avg_price,clv,customer_unique_id_cnt,total_price
year,,,,,
2016,1.226804,184.271993,226.065641,291,53623.15
2017,1.185096,166.028327,196.759486,42713,7091567.95
2018,1.175955,165.287610,194.370726,51985,8592476.42


- 연도별 CLV와 총매출을 분석한 결과, 고객생애가치(CLV)는 감소하고 있다.
- 신규고객의 유입으로 총 매출은 증가하였으나, 고객별 평균 구매빈도와 구매금액이 감소하면서 고객생애가치(CLV)가 낮아졌다.
- 재구매 가능성이 높은 고객을 찾아, 재구매 가능성이 높은 카테고리를 추천하여, 고객생애가치(CLV)를 개선해보자. 

---

### 4. RFM 지표를 활용하여 고객 분류
- RFM은 최근성(Recency) 고객의 구매빈도(Frequency) 고객의 구매금액(Monetary) 세 가지 변수를 측정한 지표
- Customer_unique_id 기준으로 RFM 지표를 산정해보자.
- RFM 지수 = a * 최근성(R) + b * 구매빈도(F) + c * 구매액(M) (abc는 각 항목에 대한 가중치)

In [10]:
# 중복되는 order_id를 제외하여 데이터 왜곡 최소화
df_rfm = pd.pivot_table(data=df, index='order_id', values=['customer_unique_id', 'purchase_date', 'total_price', 'repurchase'], 
                        aggfunc={'customer_unique_id':'last', 'purchase_date':'max', 'total_price':'sum', 'repurchase':'last'})

# RM 지표 산정에 필요한 데이터 pivot
df_rm = pd.pivot_table(data=df_rfm, index='customer_unique_id', values=['purchase_date', 'total_price'], 
                        aggfunc={'purchase_date':'max', 'total_price':'sum'})

# F 지표 산정에 필요한 데이터 pivot
df_f = pd.pivot_table(data=df_rfm, index='customer_unique_id', columns='repurchase', aggfunc='size', fill_value=0)
df_f['Frequency'] = df_f[True] + 1

# RFM 각 지표 데이터를 통합
df_rfm = pd.merge(df_rm, df_f['Frequency'], on='customer_unique_id', how='left')

# RFM 각 지표를 min-max scaler로 정규화
df_rfm['s_recency'] = (df_rfm['purchase_date'] - df_rfm['purchase_date'].min()).dt.days / (df_rfm['purchase_date'].max() - df_rfm['purchase_date'].min()).days
df_rfm['s_frequency'] = (df_rfm['Frequency'] - df_rfm['Frequency'].min()) / (df_rfm['Frequency'].max() - df_rfm['Frequency'].min())
df_rfm['s_monetary'] = (df_rfm['total_price'] - df_rfm['total_price'].min()) / (df_rfm['total_price'].max() - df_rfm['total_price'].min())

df_rfm['rev_contribution'] = df_rfm['total_price'] / df_rfm['total_price'].sum()
df_rfm['rfm_score'] = (df_rfm['s_frequency'] + df_rfm['s_recency'] + df_rfm['s_monetary'])/3

df_rfm.head()

,purchase_date,total_price,Frequency,s_recency,s_frequency,s_monetary,rev_contribution,rfm_score
customer_unique_id,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,141.90,1,0.840659,0.0,0.009690,0.000009,0.283450
0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,27.19,1,0.836538,0.0,0.001289,0.000002,0.279276
0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,86.22,1,0.255495,0.0,0.005612,0.000005,0.087036
0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,43.62,1,0.552198,0.0,0.002492,0.000003,0.184897
0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,196.89,1,0.597527,0.0,0.013717,0.000013,0.203748


In [11]:
# 선형회귀를 통해 매출기여도에 따른 스코어링이 가능한지 확인해보자.
# 선형회귀를 통해 도출된 독립변수 각 계수를 통해 각 RFM 특성이 얼마나 영향을 주는지 확인하자.

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = df_rfm[['s_recency', 's_frequency', 's_monetary']]
y = df_rfm['rev_contribution']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

# 독립변수의 각 특성에 대한 계수
coef = model.coef_
coef

array([1.78625627e-21, 4.06575815e-20, 8.67631114e-04])

- 회귀 분석을 통해 RFM 지수를 분석한 결과, 현재 최근성(R)과 구매빈도(F)가 매출기여도에 미치는 영향은 미미했다.
- 그러므로 RFM 각 Feature Score를 통해 고객을 분류하고 해당 Score에 따른 적절한 마케팅을 진행할 수 있도록 해야겠다.

---

### 5. 최종 고객별 데이터셋을 통해 머신러닝 모델 구현 테스트 진행

In [15]:
df_add = pd.pivot_table(data=df, index='customer_unique_id', values=['category', 'customer_city', 'customer_state', 'estimated_delivery_period'], 
                        aggfunc={'category':'first', 'customer_city':'last', 'customer_state':'last', 'estimated_delivery_period':'mean'})
df_add.rename(columns={'category':'first_category', 'estimated_delivery_period':'avg_esti_delivery_period'}, inplace=True)

df_final = pd.merge(df_rfm, df_add, on='customer_unique_id', how='left')
df_final['Frequency_TF'] = df_final['Frequency'] > 1
df_final.drop(columns='purchase_date', inplace=True)
df_final

,total_price,Frequency,s_recency,s_frequency,s_monetary,rev_contribution,rfm_score,first_category,customer_city,customer_state,avg_esti_delivery_period,Frequency_TF
customer_unique_id,,,,,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,141.90,1,0.840659,0.0,0.009690,0.000009,0.283450,bed_bath_table,cajamar,SP,11.0,False
0000b849f77a49e4a4ce2b2a4ca5be3f,27.19,1,0.836538,0.0,0.001289,0.000002,0.279276,health_beauty,osasco,SP,8.0,False
0000f46a3911fa3c0805444483337064,86.22,1,0.255495,0.0,0.005612,0.000005,0.087036,stationery,sao jose,SC,28.0,False
0000f6ccb0745a6a4b88665a16c9f078,43.62,1,0.552198,0.0,0.002492,0.000003,0.184897,telephony,belem,PA,32.0,False
0004aac84e0df4da2b147fca70cf8255,196.89,1,0.597527,0.0,0.013717,0.000013,0.203748,telephony,sorocaba,SP,21.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,2067.42,1,0.379121,0.0,0.150707,0.000131,0.176609,health_beauty,sanharo,PE,55.0,False
fffea47cd6d3cc0a88bd621562a9d061,84.58,1,0.633242,0.0,0.005492,0.000005,0.212911,baby,feira de santana,BA,33.0,False
ffff371b4d645b6ecea244b27531430a,112.46,1,0.212912,0.0,0.007534,0.000007,0.073482,auto,sinop,MT,45.0,False


In [34]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X = df_final[df_final['Frequency_TF'] == True][['total_price', 's_recency', 'rev_contribution', 's_monetary', 'avg_esti_delivery_period']]  # 모델 특성값
y = df_final[df_final['Frequency_TF'] == True][['Frequency']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

dt = DecisionTreeClassifier(max_depth=4, random_state=13)
dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)  
accuracy_score(y_test, y_pred)

feature_im = dt.feature_importances_
print(feature_im)

[0.29447967 0.01956983 0.14557404 0.5136481  0.02672835]
